In [ ]:
!pip install ../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/ > /dev/null # no output
!pip install torch_optimizer --no-index --find-links=file:///kaggle/input/torch-optimizer/torch_optimizer
    
import math
import pandas as pd
import os 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer as optim
from torch.optim import lr_scheduler
import torchvision.models as models
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import sklearn.model_selection
import torchvision.datasets
import torchvision.transforms as transforms
import torch
import torchvision
from PIL import Image
from sklearn.metrics import mean_squared_error
import efficientnet_pytorch
from glob import glob
import gc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
class PetDataset(torch.utils.data.Dataset):
    def __init__(self, df, data_dir, transform=None, mode='test'):
        self.df = df
        self.data_dir = data_dir
        self.transform = transform
        self.mode = mode
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.df.iloc[idx, 13]
        dense_features = self.df.iloc[idx, 1:13]
        img = Image.open(img_name)
        
        if self.transform is not None:
            img = self.transform(img)
        dense_features = torch.tensor(dense_features)
        return img, dense_features

In [ ]:
test_data = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")

test_images_path = []
for dirpath, dirnames, filenames in os.walk("../input/petfinder-pawpularity-score/test"):
    for filename in [f for f in filenames if f.endswith(".jpg")]:
        test_images_path.append(os.path.join(dirpath, filename))

test_data['input_file_loc'] = sorted(test_images_path)
test_data

In [ ]:
class PawpularModel(nn.Module):
    def __init__(self, depth):
        super(PawpularModel, self).__init__()
        self.depth = depth
        self.base = efficientnet_pytorch.EfficientNet.from_name(f'efficientnet-b{self.depth}' )
        pretrained_file = glob(f'../input/efficientnet-pytorch/efficientnet-b{self.depth}*')[0]
        checkpoint = torch.load(pretrained_file)
        self.base.load_state_dict(checkpoint)
#         self.model = timm.create_model(model_name, pretrained=False, in_chans=3)
        self.base._fc = nn.Linear(self.base._fc.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(140, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, image, features):
        x = self.base(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
super_final_predictions = []

for fold_ in range(10):
    model = PawpularModel(0)
    model.load_state_dict(torch.load(f"../input/models-fold10/fold_{fold_}.pth"))
    model.cuda()
#     model.eval()
#     model.load(f"../input/pawpular-models/model_f{fold_}.bin", device="cuda", weights_only=True)
    dataset  = PetDataset(df=test_data, data_dir="../input/petfinder-pawpularity-score/test", transform=transform, mode='test')
    test_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=0)
    #dataiter = iter(test_loader)
    #images, labels = dataiter.next()
#     print(images.shape, labels.shape)
#     print(labels)

    final_test_predictions = []
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, features = data
            features = features.cuda()
            inputs = inputs.cuda()
            outputs = model(inputs, features)
            outputs = outputs.detach().cpu().numpy()
            #print(outputs)
            for preds in outputs:
                final_test_predictions.extend(preds.ravel().tolist())
#         print(final_test_predictions)
    super_final_predictions.append(final_test_predictions)

print("Finishing")
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
test_data["Pawpularity"] = super_final_predictions
test_data = test_data[["Id", "Pawpularity"]]
test_data.to_csv("submission.csv", index=False)